<a href="https://colab.research.google.com/github/rohitkr8527/MediGuide-Llama/blob/main/Mediguide_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!nvidia-smi

Sun Jun 22 23:38:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             28W /   70W |    5646MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from huggingface_hub import login

login()

In [4]:
!pip install trl==0.11.4 transformers==4.45.0 peft==0.13.2 datasets==2.18.0 accelerate==0.27.2 bitsandbytes==0.43.1 torch>=2.1.0 triton==3.1.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.2.0 which is incompatible.


In [4]:
!pip list | grep -E 'trl|transformers|peft|datasets|accelerate|bitsandbytes|torch|triton'

accelerate                            0.27.2
bitsandbytes                          0.43.1
datasets                              2.18.0
fastrlock                             0.8.3
peft                                  0.13.2
sentence-transformers                 4.1.0
tensorflow-datasets                   4.9.9
torch                                 2.5.1
torchao                               0.10.0
torchaudio                            2.6.0+cu124
torchdata                             0.11.0
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.21.0+cu124
transformers                          4.45.0
triton                                3.1.0
trl                                   0.11.4
vega-datasets                         0.9.0


In [3]:
import json
from datasets import Dataset

with open("HealthCareMagic-100k.json", "r", encoding='utf-8') as f:
    raw_data = json.load(f)

dataset = Dataset.from_list([
    {
        "prompt": f"### Instruction:\n{entry['instruction']}\n\n### Input:\n{entry['input']}\n\n### Response:\n",
        "response": entry["output"]
    } for entry in raw_data
])

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",

)

tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [17]:
model.config.use_cache = False

In [6]:
!nvidia-smi

Sun Jun 22 23:38:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             28W /   70W |    5646MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [20]:
from transformers import DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(
        example["prompt"] + example["response"],
        truncation=True,
        max_length=512,
        padding="longest"
    )


tokenized_dataset = dataset.map(tokenize, remove_columns=["prompt", "response"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [25]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mistral-lora-e-doctor",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=500,
    save_total_limit=1,
    fp16=True,
    report_to="none"
)

# STEP 6: Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [24]:
import warnings
warnings.filterwarnings("ignore", message=".*use_reentrant parameter should be passed explicitly.*")

In [ ]:
trainer.train()

In [29]:
model.save_pretrained("./mistral-lora-e-doctor")
tokenizer.save_pretrained("./mistral-lora-e-doctor")

('./mistral-lora-e-doctor/tokenizer_config.json',
 './mistral-lora-e-doctor/special_tokens_map.json',
 './mistral-lora-e-doctor/tokenizer.json')

In [30]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [32]:
model.save_pretrained('/content/drive/My Drive/mistral-lora-e-doctor')
tokenizer.save_pretrained('/content/drive/My Drive/mistral-lora-e-doctor')


('/content/drive/My Drive/mistral-lora-e-doctor/tokenizer_config.json',
 '/content/drive/My Drive/mistral-lora-e-doctor/special_tokens_map.json',
 '/content/drive/My Drive/mistral-lora-e-doctor/tokenizer.json')